In [ ]:
K_1 = 1
K_2 = 1

m_1 = 1
m_2 = 1

U = 1

de

def findv0(K, m):
    v_0 = np.abs(np.sqrt(2*K/m))
    return v_0



$$U_\mathrm{eff}(r) = \frac 12 \frac{l^2}{\mu r^2} + U(r) \qquad \text{and} U(r) = \frac k r$$